<a href="https://colab.research.google.com/github/kwb425/class-2025-spring/blob/main/class-2025-spring_0530-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    !pip install datasets huggingface_hub


### Data


raw
	In the latest AP dispatch from Frankfurt, which details how the European Union's chief trade negotiator said he had “good calls” with Trump administration officials on Monday, David McHugh noted that the EU "has offered Trump a 'zero for zero' deal in which tariffs would be removed on industrial goods including automobiles, but the U.S. administration has said it will not lower tariffs below a 10% baseline imposed on almost all its trading partners. Trump has also announced tariffs of 25% on steel and automobiles."
	That raises the question of where the negotiations, which got back on the rails over the weekend after a call between President Trump and EU Commission President Ursula von der Leyen, will go from here — and what Trump will have to say next time he discusses the $1.8 trillion trade relationship.
	...
pretrain
	one record:
		In the latest AP dispatch from Frankfurt, which details how the European Union's chief trade negotiator said he had “good calls” with Trump administration officials on Monday, David McHugh noted that the EU "has offered Trump a 'zero for zero' deal in which tariffs would be removed on industrial goods including automobiles, but the U.S. administration has said it will not lower tariffs below a 10% baseline imposed on almost all its trading partners. Trump has also announced tariffs of 25% on steel and automobiles."
		That raises the question of where the negotiations, which got back on the rails over the weekend after a call between President Trump and EU Commission President Ursula von der Leyen, will go from here — and what Trump will have to say next time he discusses the $1.8 trillion trade relationship.
		...



instructional - alpaca
	[
		{
			"Instruction": "Task we want the model to perform. 11111",
			"Input": "Optional, but useful, it will essentially be the user's query. 11111",
			"Output": "The expected result of the task and the output of the model 11111."
		},
		{
			"Instruction": "Task we want the model to perform. 22222",
			"Input": "Optional, but useful, it will essentially be the user's query. 22222",
			"Output": "The expected result of the task and the output of the model. 22222"
		},
	]
full fine-tuning | Lora fine-tuning
	one record:
		"""Below is an instruction that describes a task, paired with an input that provides further context.
		Write a response that appropriately completes the request.

		### Instruction:
		Task we want the model to perform. 11111

		### Input:
		Optional, but useful, it will essentially be the user's query. 11111

		### Response:
		The expected result of the task and the output of the model. 11111"""
	one record:
		"""Below is an instruction that describes a task, paired with an input that provides further context.
		Write a response that appropriately completes the request.

		### Instruction:
		Task we want the model to perform. 22222

		### Input:
		Optional, but useful, it will essentially be the user's query. 22222

		### Response:
		The expected result of the task and the output of the model. 22222"""



conversational - shareGPT
{
	"conversations": [
		{
			"from": "system",
			"value": "You are a helpful assistant."
		},
		{
			"from": "human",
			"value": "Can you help me make pasta carbonara?"
		},
		{
			"from": "gpt",
			"value": "Would you like the traditional Roman recipe, or a simpler version?"
		},
		{
			"from": "human",
			"value": "The traditional version please"
		},
		{
			"from": "gpt",
			"value": "The authentic Roman carbonara uses just a few ingredients: pasta, guanciale, eggs, Pecorino Romano, and black pepper. Would you like the detailed recipe?"
		}
	]
}
full fine-tuning | Lora fine-tuning
	one record:
		<|im_start|>system
		You are a helpful assistant.<|im_end|>
		<|im_start|>user
		Can you help me make pasta carbonara?<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		Would you like the traditional Roman recipe, or a simpler version?<|im_end|>
		<|im_start|>user
		The traditional version please<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		The authentic Roman carbonara uses just a few ingredients: pasta, guanciale, eggs, Pecorino Romano, and black pepper. Would you like the detailed recipe?<|im_end|>



conversational - ChatML
{
	"messages": [
		{
			"role": "system",
			"content": "You are a helpful assistant."
		},
		{
			"role": "user",
			"content": "What is 1+1?"
		},
		{
			"role": "assistant",
			"content": "It's 2!"
		},
		{
			"role": "user",
			"content": "What is 2+2?"
		},
		{
			"role": "assistant",
			"content": "It's 4!"
		}
	]
}
full fine-tuning | Lora fine-tuning
	one record:
		<|im_start|>system
		You are a helpful assistant.<|im_end|>
		<|im_start|>user
		What is 1+1?<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		It's 2!<|im_end|>
		<|im_start|>user
		What is 2+2?<|im_end|>
		<|im_start|>assistant
		<think>
		</think>
		It's 4!<|im_end|>



In [ ]:
from datasets import Dataset
from huggingface_hub import login
from huggingface_hub import create_repo, delete_repo
import json


# login(token="key")

with open('class-2025-spring_0530-0.json', 'r') as f:
    loaded_data = json.load(f)

non_reasoning_dataset = Dataset.from_list(loaded_data)
# delete_repo(repo_id="userID/2025-spring-conversations", repo_type="dataset")
# create_repo(repo_id="userID/2025-spring-conversations", repo_type="dataset", private=False, exist_ok=True)
# non_reasoning_dataset.push_to_hub("userID/2025-spring-conversations")
# from datasets import load_dataset
# non_reasoning_dataset = load_dataset("userID/2025-spring-conversations", split = "train")



### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

# https://huggingface.co/unsloth
fourbit_models = [
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False,
)

In [ ]:
# LoRA adapters, updating 1 to 10% of all parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # rank, 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # Rank stabilized LoRA
    loftq_config = None,  # LoftQ
)

<a name="Data"></a>
### Data Check


In [ ]:
non_reasoning_dataset

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(non_reasoning_dataset)

non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

In [ ]:
non_reasoning_conversations[0]

In [ ]:
import pandas as pd
non_reasoning_subset = pd.Series(non_reasoning_conversations)

In [ ]:
data = pd.concat([
    pd.Series(non_reasoning_subset)
])
data.name = "text"

from datasets import Dataset
non_reasoning_dataset = Dataset.from_pandas(pd.DataFrame(data))
non_reasoning_dataset = non_reasoning_dataset.shuffle(seed = 3407)

<a name="Train"></a>
### Train the model


In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = non_reasoning_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

<a name="Inference"></a>
### Inference
According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`
For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
messages = [
    {"role" : "user", "content" : "자유심증주의란 무엇인가요? 어떻게 적용되나요? 헷갈리는데?"},
    # {"role" : "user", "content" : "경복궁이 뭐에요?"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

In [ ]:
# messages = [
#     {"role" : "user", "content" : "자백의 보강법칙이란? 무엇인가요? 알려주세요!"}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize = False,
#     add_generation_prompt = True, # Must add for generation
#     enable_thinking = True, # Disable thinking
# )

# from transformers import TextStreamer
# _ = model.generate(
#     **tokenizer(text, return_tensors = "pt").to("cuda"),
#     max_new_tokens = 1024, # Increase for longer outputs!
#     temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
#     streamer = TextStreamer(tokenizer, skip_prompt = True),
# )

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

In [ ]:
# model.save_pretrained("my_lora_model")  # Local saving
# tokenizer.save_pretrained("my_lora_model")
model.push_to_hub("userId/my_lora_model", token = "key") # Online saving
tokenizer.push_to_hub("userId/my_lora_model", token = "key") # Online saving

Loading trained model

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "userID/my_lora_model",
    max_seq_length = 2048,
    load_in_4bit = True,
)

### Saving to float16 for VLLM

Select `merged_16bit` for float16 or `merged_4bit` for int4.

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "key")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "key")

# Just LoRA adapters
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "key")

### GGUF / llama.cpp Conversion
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")
